In [8]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [9]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [10]:
import random
random.seed(42)

from src.utils import TEST_KEY_MAPPING

test_q_keys_mapping = TEST_KEY_MAPPING

# test_q_keys_mapping = {
#     "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
#     "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
#     "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
#     "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
#     "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
# }


# Hyper parameters

In [11]:

survey_names = [
    # "American_Trends_Panel_W26",
    # "American_Trends_Panel_W27",
    # "American_Trends_Panel_W29",
    # "American_Trends_Panel_W32",
    "American_Trends_Panel_W34",
    "American_Trends_Panel_W41",
    "American_Trends_Panel_W82",
]

def skip_exp(filename):

    # check known test
    if 'v17' not in filename or all(['0904' not in filename, '0905' not in filename]): return True
    # if 'history_demo' in filename and 'history_demo_persona' not in filename: return True
    if 'bdemo' not in filename: return True

    return False

def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    

    segs = ['history', 'demo', 'persona', 'withname', 'all' 'allq', 'ragupper', 'bdemo']
    for level in ['13', '16', '17']: segs.append('v'+level)
    for train in ['200', '500', '2000', 'All']: segs.append('train'+train)
    for top in range(10, 110, 10): segs.append('top'+str(top))
    for skew in ['5', '3', '10']: segs.append('skew'+skew)
    for usedemo in ['True', 'False']: segs.append('usedemo'+usedemo)
    for run in ['run1', 'run2', 'run3']: segs.append(run)
    for setting in ['queryponly', 'querydp', 'querydonly']: segs.append(setting)

    exp_elements = [_ for _ in filename.split('_') if _ in segs]
    final = '_'.join(exp_elements)
    return final

# Main

In [12]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name, sort_by):

    with open(f'experiment/data/human_resp/{survey_name}/user_test_q_key_mapping.json', 'r') as f:
        user_test_q_key_mapping = json.load(f)
    
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]

    for key in res:
        print(key, len(res[key]))

    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    # acc_df.sort_values(by='acc', ascending=False)


    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in survey_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        print(filename)
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = user_test_q_key_mapping[str(entry["user_idx"])][q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    print(acc_df)
    print(q_wd_df)

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    if sort_by == 'q_1wd':
        merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    elif sort_by == 'acc':
        merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df

    return merged_df

# Results

In [13]:
from IPython.display import display

for survey_name in survey_names:
    print(survey_name)
    sort_by = 'acc'
    # sort_by = 'q_1wd'
    merged = get_full_results(survey_name, sort_by=sort_by)
    display(merged)
    print('\n\n\n')

American_Trends_Panel_W34
v17_bdemo_history_demo_persona_run1 2
v17_bdemo_history_run1 2
v17_bdemo_history_demo_run2 2
v17_bdemo_history_demo_run3 2
v17_bdemo_history_demo_persona_run2 2
v17_bdemo_history_run2 2
v17_bdemo_history_demo_run1 2
v17_bdemo_history_run3 2
v17_bdemo_history_demo_persona_run3 2
date0905_v17_personadb_surveys_bdemo_American_Trends_Panel_W34_haikupred_prompt3_history_demo_persona_run1.json
Prediction: |not sure|, Gold: |neither better nor worse for your health than foods with no genetically modified ingredients|, References: |Better for your health than foods with no genetically modified ingredients/Worse for your health than foods with no genetically modified ingredients/Neither better nor worse for your health than foods with no genetically modified ingredients/Refused|
Prediction: |humans have evolved over time due to processes that were guided or allowed by god or a higher power|, Gold: |humans have evolved over time due to processes such as natural selectio

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_history_demo_persona_run1,'=593/1000,0.593,"{1.0: 352, 0.0: 593, 2.0: 12, 3.0: 3, 0.5: 7}",0.401758,20,13
v17_bdemo_history_demo_persona_run3,'=592/1000,0.592,"{1.0: 353, 0.0: 592, 2.0: 12, 3.0: 3, 0.5: 7}",0.402792,20,13
v17_bdemo_history_demo_persona_run2,'=589/1000,0.589,"{1.0: 355, 0.0: 589, 2.0: 12, 3.0: 3, 0.5: 8}",0.405377,20,13
v17_bdemo_history_demo_run3,'=589/1000,0.589,"{1.0: 360, 0.0: 588, 2.0: 11, 3.0: 4, 0.5: 9}",0.409979,15,13
v17_bdemo_history_demo_run1,'=585/1000,0.585,"{1.0: 362, 0.0: 584, 2.0: 13, 3.0: 3, 0.5: 10}",0.41358,15,13
v17_bdemo_history_run1,'=585/1000,0.585,"{1.0: 357, 0.0: 584, 2.0: 18, 3.0: 2, 0.5: 9}",0.415979,17,13
v17_bdemo_history_run3,'=585/1000,0.585,"{1.0: 355, 0.0: 584, 2.0: 17, 3.0: 3, 0.5: 10}",0.415893,18,13
v17_bdemo_history_demo_run2,'=584/1000,0.584,"{1.0: 364, 0.0: 583, 2.0: 13, 3.0: 3, 0.5: 8}",0.415036,16,13
v17_bdemo_history_run2,'=581/1000,0.581,"{1.0: 359, 0.0: 580, 2.0: 18, 3.0: 3, 0.5: 10}",0.421649,17,13






American_Trends_Panel_W41
v17_bdemo_history_run1 2
v17_bdemo_history_demo_persona_run3 2
v17_bdemo_history_demo_run1 2
v17_bdemo_history_demo_run3 2
v17_bdemo_history_run2 2
v17_bdemo_history_demo_run2 2
v17_bdemo_history_demo_persona_run1 2
v17_bdemo_history_run3 2
v17_bdemo_history_demo_persona_run2 2
date0905_v17_personadb_surveys_bdemo_American_Trends_Panel_W41_haikupred_prompt3_history_run1.json
Prediction: |outside of a professional or work setting|, Gold: |yes|, References: |Yes/No/Refused|
date0905_v17_personadb_surveys_bdemo_American_Trends_Panel_W41_haikupred_prompt3_history_demo_persona_run3.json
Prediction: |maintained at current levels|, Gold: |an important, but not a top priority|, References: |A top priority/An important, but not a top priority/A lower priority/Should not be done/Refused|
Prediction: |both|, Gold: |yes|, References: |Yes/No/Refused|
Prediction: |not at all worried|, Gold: |a lower priority|, References: |A top priority/An important, but not a top pri

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_history_demo_persona_run1,'=517/1000,0.517,"{1.0: 328, 0.0: 529, 0.5: 72, 2.0: 40, 1.5: 5,...",0.471341,7,16
v17_bdemo_history_demo_persona_run2,'=514/1000,0.514,"{1.0: 330, 0.0: 526, 0.5: 72, 2.0: 42, 1.5: 5,...",0.476994,6,16
v17_bdemo_history_demo_persona_run3,'=513/1000,0.513,"{1.0: 331, 0.0: 525, 0.5: 75, 2.0: 40, 1.5: 5,...",0.474974,5,16
v17_bdemo_history_demo_run2,'=513/1000,0.513,"{0.0: 526, 1.0: 332, 0.5: 79, 2.0: 40, 1.5: 2,...",0.469419,3,16
v17_bdemo_history_demo_run1,'=509/1000,0.509,"{1.0: 335, 0.5: 79, 0.0: 522, 2.0: 40, 1.5: 3,...",0.474006,3,16
v17_bdemo_history_demo_run3,'=504/1000,0.504,"{1.0: 336, 0.5: 80, 0.0: 517, 2.0: 41, 1.5: 3,...",0.47855,5,16
v17_bdemo_history_run3,'=499/1000,0.499,"{0.0: 512, 1.0: 334, 0.5: 83, 2.0: 52, 3.0: 2,...",0.494919,0,16
v17_bdemo_history_run1,'=495/1000,0.495,"{0.0: 508, 1.0: 339, 0.5: 84, 2.0: 49, 3.0: 2,...",0.494914,1,16
v17_bdemo_history_run2,'=491/1000,0.491,"{0.0: 504, 1.0: 341, 0.5: 86, 2.0: 50, 3.0: 2,...",0.499492,0,16






American_Trends_Panel_W82
v17_bdemo_history_demo_run1 2
v17_bdemo_history_run3 2
v17_bdemo_history_demo_persona_run1 2
v17_bdemo_history_run1 2
v17_bdemo_history_demo_run2 2
v17_bdemo_history_demo_persona_run3 2
v17_bdemo_history_run2 2
v17_bdemo_history_demo_persona_run2 2
v17_bdemo_history_demo_run3 2
date0905_v17_personadb_surveys_bdemo_American_Trends_Panel_W82_haikupred_prompt3_history_demo_run1.json
Prediction: |very confident|, Gold: |a lot of confidence|, References: |A lot of confidence/Some confidence/Not too much confidence/No confidence at all/Refused|
Prediction: |very confident|, Gold: |a lot of confidence|, References: |A lot of confidence/Some confidence/Not too much confidence/No confidence at all/Refused|
date0905_v17_personadb_surveys_bdemo_American_Trends_Panel_W82_haikupred_prompt3_history_run3.json
Prediction: |in foreign policy, the us should take into account the interests of its allies even if it means making compromises with them|, Gold: |in foreign policy

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_history_demo_persona_run2,'=580/1000,0.58,"{0.0: 576, 1.0: 349, 2.0: 35, 0.5: 11, 3.0: 3}",0.445072,7,19
v17_bdemo_history_demo_persona_run1,'=579/1000,0.579,"{0.0: 575, 1.0: 350, 2.0: 35, 0.5: 11, 3.0: 3}",0.446099,7,19
v17_bdemo_history_demo_run3,'=573/1000,0.573,"{0.0: 571, 1.0: 358, 2.0: 36, 0.5: 8, 3.0: 5}",0.4591,3,19
v17_bdemo_history_run2,'=571/1000,0.571,"{0.0: 570, 1.0: 349, 2.0: 38, 0.5: 10, 3.0: 4}",0.455201,10,19
v17_bdemo_history_demo_persona_run3,'=570/1000,0.57,"{0.0: 567, 1.0: 359, 2.0: 35, 0.5: 11, 3.0: 3}",0.454872,6,19
v17_bdemo_history_demo_run1,'=570/1000,0.57,"{0.0: 568, 1.0: 361, 2.0: 36, 0.5: 9, 3.0: 5}",0.462206,2,19
v17_bdemo_history_demo_run2,'=568/1000,0.568,"{0.0: 566, 1.0: 361, 2.0: 36, 0.5: 8, 3.0: 5}",0.463115,5,19
v17_bdemo_history_run3,'=567/1000,0.567,"{0.0: 566, 1.0: 350, 2.0: 40, 0.5: 11, 3.0: 4}",0.460865,10,19
v17_bdemo_history_run1,'=561/1000,0.561,"{0.0: 560, 1.0: 361, 2.0: 38, 0.5: 10, 3.0: 4}",0.466598,8,19


In [14]:
print(merged.to_csv())

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_bdemo_history_demo_persona_run2,'=580/1000,0.58,"Counter({0.0: 576, 1.0: 349, 2.0: 35, 0.5: 11, 3.0: 3})",0.4450718685831622,7,19
v17_bdemo_history_demo_persona_run1,'=579/1000,0.579,"Counter({0.0: 575, 1.0: 350, 2.0: 35, 0.5: 11, 3.0: 3})",0.44609856262833675,7,19
v17_bdemo_history_demo_run3,'=573/1000,0.573,"Counter({0.0: 571, 1.0: 358, 2.0: 36, 0.5: 8, 3.0: 5})",0.4591002044989775,3,19
v17_bdemo_history_run2,'=571/1000,0.571,"Counter({0.0: 570, 1.0: 349, 2.0: 38, 0.5: 10, 3.0: 4})",0.4552008238928939,10,19
v17_bdemo_history_demo_persona_run3,'=570/1000,0.57,"Counter({0.0: 567, 1.0: 359, 2.0: 35, 0.5: 11, 3.0: 3})",0.45487179487179485,6,19
v17_bdemo_history_demo_run1,'=570/1000,0.57,"Counter({0.0: 568, 1.0: 361, 2.0: 36, 0.5: 9, 3.0: 5})",0.46220633299284986,2,19
v17_bdemo_history_demo_run2,'=568/1000,0.568,"Counter({0.0: 566, 1.0: 361, 2.0: 36, 0.5: 8, 3.0: 5})",0.46311475409836067,5,19
v17_bdemo_history_run3,'=567/1000,0.567,"Cou